# Satellite Data Processing 
  
Katie Brennan  
April 2, 2021  
  
This notebook calculates the monthly and annual slopes and R$^2$ values between temperature and sea ice extent observations. Temperature data is taken from ERA5 and total Arctic sea ice extent is calculated from two versions of satellite data: NASA Team and Bootstrap. Arctic mean temperature north of 60 and global mean temperatures are both considered. 

In [ ]:
import xarray as xr
import scipy.stats as stats 
import matplotlib.pyplot as plt
import os
import numpy as np

from importlib import reload 

In [ ]:
import sys 
sys.path.append("../local_analysis/")

import preprocess_utils as pputils
import analysis_utils as autils 

In [ ]:
all_months = [1,2,3,4,5,6,7,8,9,10,11,12]
month_names = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec','Annual']

### Load ERA5 temperature data: 

In [ ]:
obs_dir = '/glade/work/mkbren/cmip6-data/cmip6-preprocessed/observations/'
obs_name = 'tas_ERA5_pan_Arctic.nc'

era_data = xr.load_dataset(os.path.join(obs_dir,obs_name))

### Load sea ice satellite data: 

In [ ]:
# # SIA all three products: 
# sia_path = '/glade/work/lettier/archived_data/202003_data_paper_antarctic_sia_cmip6/processed_OBS_sia_1979-2018.nc'

# #SIC NASA Team and Bootstrap monthly concentration fields here: 
# path_sat_sic = '/glade/work/lettier/CDR/mon/seaice_conc_monthly_nh_1979-2018.nc'

#SIA and SIE for NASA Team and Bootstrap, Arctic and Antarctic here: 
path_sat_nt_bt = '/glade/work/lettier/CDR/mon/processed/sia_cdr_monthly_processed_nhsh_1979-2018.nc'

sat_nt_bt = xr.open_dataset(path_sat_nt_bt)

sat_nt_bt.sie_nh_bt[np.where(sat_nt_bt.sie_nh_bt<0.2e7)] = np.nan

sat_nt_bt['sie_nh_bt_anom'] = sat_nt_bt.sie_nh_bt.groupby('time.month') - sat_nt_bt.sie_nh_bt.groupby('time.month').mean()
sat_nt_bt['sie_nh_nt_anom'] = sat_nt_bt.sie_nh_nt.groupby('time.month') - sat_nt_bt.sie_nh_nt.groupby('time.month').mean()

#### Select years that overlap with Historical simulations (1979-2014): 

In [ ]:
sat_nt_bt_79_14 = sat_nt_bt.sel(time=slice('1979-01-01','2014-12-01'))
sat_nt_bt_79_14

### Slopes relative to Arctic mean temperature (60N): 

In [ ]:
era_data_79_14 = era_data.sel(time=slice('1979-01-01','2014-12-01'))

[slopes_obs_nt_am_79_14, 
r_obs_nt_am_79_14,
intercept_obs_nt_am_79_14] = autils.scatter_linreg(era_data_79_14['t2m_arc_mean_anom60'],sat_nt_bt.sie_nh_nt_anom*1e-6,
                                                 era_data_79_14,all_months,'obs',plotflag=False)

[slopes_obs_bt_am_79_14, 
r_obs_bt_am_79_14,
intercept_obs_bt_am_79_14] = autils.scatter_linreg(era_data_79_14['t2m_arc_mean_anom60'],sat_nt_bt.sie_nh_bt_anom*1e-6,
                                                 era_data_79_14,all_months,'obs',plotflag=False)

In [ ]:
plt.plot(month_names[:-1],slopes_obs_bt_am_79_14, label='bootstrap')
plt.plot(month_names[:-1],slopes_obs_nt_am_79_14, label='NASA team')
plt.legend()
plt.title('Arctic mean tas (60N) vs Total SIE')

In [ ]:
plt.plot(month_names[:-1],np.power(r_obs_bt_am_79_14,2), label='bootstrap')
plt.plot(month_names[:-1],np.power(r_obs_nt_am_79_14,2), label='NASA team')
plt.legend()
plt.title('Arctic mean tas (60N) vs Total SIE')

### Slopes relative to global mean temperature: 

In [ ]:
era_data_79_14 = era_data.sel(time=slice('1979-01-01','2014-12-01'))

[slopes_obs_nt_gm_79_14, 
r_obs_nt_gm_79_14,
intercept_obs_nt_gm_79_14] = autils.scatter_linreg(era_data_79_14['t2m_gm_anom'],sat_nt_bt_79_14.sie_nh_nt_anom*1e-6,
                                                 era_data_79_14,all_months,'obs',plotflag=False)

[slopes_obs_bt_gm_79_14, 
r_obs_bt_gm_79_14,
intercept_obs_bt_gm_79_14] = autils.scatter_linreg(era_data_79_14['t2m_gm_anom'],sat_nt_bt_79_14.sie_nh_bt_anom*1e-6,
                                                 era_data_79_14,all_months,'obs',plotflag=False)

In [ ]:
plt.plot(month_names[:-1],slopes_obs_bt_gm_79_14, label='bootstrap')
plt.plot(month_names[:-1],slopes_obs_nt_gm_79_14, label='NASA team')
plt.legend()
plt.title('GMT vs Total SIE')

In [ ]:
plt.plot(month_names[:-1],np.power(r_obs_bt_gm_79_14,2), label='bootstrap')
plt.plot(month_names[:-1],np.power(r_obs_nt_gm_79_14,2), label='NASA team')
plt.legend()
plt.title('GMT vs Total SIE')

### Annual slopes: 

In [ ]:
slopes_obs_bt_gm_79_14_annual,_,r_obs_bt_gm_79_14_annual,_,_ = stats.linregress(era_data_79_14['t2m_gm_annual_anom'][:-1],
                                                                                 sat_nt_bt.sie_nh_bt_anom.groupby('time.year').mean()*1e-6)

slopes_obs_nt_gm_79_14_annual,_,r_obs_nt_gm_79_14_annual,_,_ = stats.linregress(era_data_79_14['t2m_gm_annual_anom'][:-1],
                                                                                 sat_nt_bt.sie_nh_nt_anom.groupby('time.year').mean()*1e-6)

slopes_obs_bt_am_79_14_annual,_,r_obs_bt_gm_79_14_annual,_,_ = stats.linregress(era_data_79_14['t2m_arc_mean_annual_anom60'][:-1],
                                                                                 sat_nt_bt.sie_nh_bt_anom.groupby('time.year').mean()*1e-6)

slopes_obs_nt_am_79_14_annual,_,r_obs_nt_gm_79_14_annual,_,_ = stats.linregress(era_data_79_14['t2m_arc_mean_annual_anom60'][:-1],
                                                                                 sat_nt_bt.sie_nh_nt_anom.groupby('time.year').mean()*1e-6)

slopes_obs_bt_gm_79_14.append(slopes_obs_bt_gm_79_14_annual)
slopes_obs_nt_gm_79_14.append(slopes_obs_nt_gm_79_14_annual)
slopes_obs_bt_am_79_14.append(slopes_obs_bt_am_79_14_annual)
slopes_obs_nt_am_79_14.append(slopes_obs_nt_am_79_14_annual)